# Standard Name Table

A so-called "standard name table" defines "standard names", which is a concept used by the [CF Convention](https://cfconventions.org/).

Those standard names are used to define the meaning of a numerical variable in files (typically netCDF4 files).

With this library, we can describe a standard name table using JSON-LD. **Note**, that only a simplified version of the original CF Conventions is modelled!

This notebook walks you through the main steps of building such a table yourself using Python:

In [1]:
import ssnolib
from ssnolib.namespace import SSNO
from ssnolib.prov import Person, Organization

Let's start by instantiate a table class. We add a title and a responsible person:

In [2]:
snt = ssnolib.StandardNameTable(
    title='SNT from scratch',
    version='v1',
    creator=[
        Person(first_name="Matthias", last_name="Probst", orcidID="https://orcid.org/0000-0001-8729-0482", id="https://orcid.org/0000-0001-8729-0482"),
        Person(first_name="John", last_name="Doe", ),
    ]
)

In [3]:
print(snt.model_dump_jsonld())

{
    "@context": {
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "dcat": "http://www.w3.org/ns/dcat#",
        "dcterms": "http://purl.org/dc/terms/",
        "prov": "http://www.w3.org/ns/prov#",
        "ssno": "https://matthiasprobst.github.io/ssno#",
        "foaf": "http://xmlns.com/foaf/0.1/",
        "m4i": "http://w3id.org/nfdi4ing/metadata4ing#",
        "schema": "https://schema.org/"
    },
    "@type": "ssno:StandardNameTable",
    "dcterms:title": "SNT from scratch",
    "dcterms:creator": [
        {
            "@type": "prov:Person",
            "foaf:firstName": "Matthias",
            "foaf:lastName": "Probst",
            "orcidID": "https://orcid.org/0000-0001-8729-0482",
            "@id": "https://orcid.org/0000-0001-8729-0482"
        },
        {
            "@type": "prov:Person",
            "foaf:firstName": "John",
            "foaf:lastName": "Doe",
            "@id": "_:N639fb8a5f68048bc9

C:\Users\matth\anaconda3\envs\ssnolib\lib\site-packages\pydantic\main.py:347: UserWarning: Pydantic serializer warnings:
  Expected `list[Union[definition-ref, Transformation]]` but got `dict` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


Let's add some standard names to the table:

In [4]:
snt.standardNames = [
    ssnolib.StandardName(
        standard_name="air_density",
        description="The density of air",
        canonicalUnits="kg/m^3"
    ),
    ssnolib.StandardName(
        standard_name="coordinate",
        description="The spatial coordinate vector.",
        canonicalUnits="m"
    ),
    ssnolib.StandardName(
        standard_name="velocity",
        description="The velocity vector of an object or fluid.",
        canonicalUnits="m/s"
    )
]

So far we only have two standard names. We can define modification rules, to build new, verified standard names. For example, "x_velocity" would be a reasonable new standard name for the table.

So let's define such a modification rule. We call it a `Qualification`. The one we would like to define should be used directly of an already existing standard name, e.g. "SSNO:AnyStandardName":

In [5]:
component = ssnolib.Qualification(
    name="component",
    hasValidValues=["x", "y", "z"],
    description="The component of a vector",
    before=SSNO.AnyStandardName
)

transformation = ssnolib.Transformation(
    name="C_derivative_of_X",
    description="derivative of X with respect to distance in the component direction, which may be x, y or z.",
    altersUnit="[X]/[C]",
    hasCharacter=[
        ssnolib.Character(character="X", associatedWith=SSNO.AnyStandardName),
        ssnolib.Character(character="C", associatedWith=component.id),
    ]
)

Add it to the SNT:

In [6]:
snt.hasModifier = [component, transformation]

We can check standard name strings, whether they apply to the modification rule:

In [7]:
snt.verify_name("vertical_velocity")

False

In [8]:
snt.verify_name("x_velocity")

True

In [9]:
snt.verify_name("x_component")

False

Also, adding new standard names can go through a verification:

In [10]:
snt.add_new_standard_name("x_coordinate", verify=True) # verify=False will just add the standard name and interpret it as a core standard name

StandardName(id=_:Ncc55ad2e36394c85bcc06b7040abb9e8, standardName=x_coordinate, canonicalUnits=http://qudt.org/vocab/unit/UNITLESS, description=N.A)

## Export standard name tables
We can export to JSON-LD and also generate an HTML file:

Export the current version of the standard name table to JSON-LD:

In [18]:
print(snt.model_dump_jsonld())

{
    "@context": {
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "dcat": "http://www.w3.org/ns/dcat#",
        "dcterms": "http://purl.org/dc/terms/",
        "prov": "http://www.w3.org/ns/prov#",
        "ssno": "https://matthiasprobst.github.io/ssno#",
        "foaf": "http://xmlns.com/foaf/0.1/",
        "m4i": "http://w3id.org/nfdi4ing/metadata4ing#",
        "schema": "http://schema.org/",
        "skos": "http://www.w3.org/2004/02/skos/core#"
    },
    "@type": "ssno:StandardNameTable",
    "dcterms:title": "SNT from scratch",
    "dcterms:creator": [
        {
            "@type": "prov:Person",
            "foaf:firstName": "Matthias",
            "foaf:lastName": "Probst",
            "orcidID": "https://orcid.org/0000-0001-8729-0482",
            "@id": "https://orcid.org/0000-0001-8729-0482"
        },
        {
            "@type": "prov:Person",
            "foaf:firstName": "John",
            "foaf:las

Write the HTML file:

In [17]:
snt.to_html(folder="tmp")

WindowsPath('tmp/SNT from scratch.html')